In [1]:
import solcx
from solcx import compile_standard, install_solc
import json
import os

# Install the required Solidity compiler version
solc_version = '0.8.17'
try:
    solcx.install_solc(solc_version)
except Exception as e:
    print(f"Solidity compiler installation error: {e}")

# Function to compile a single contract
def compile_contract(contract_file, contract_name):
    with open(contract_file, "r") as file:
        source = file.read()

    # Prepare sources dictionary
    sources = {
        contract_file: {
            "content": source
        }
    }

    # Compile the contract
    compiled_sol = compile_standard(
        {
            "language": "Solidity",
            "sources": sources,
            "settings": {
                "outputSelection": {
                    "*": {
                        "*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]
                    }
                }
            },
        },
        solc_version=solc_version
    )

    # Extract ABI and Bytecode
    try:
        abi = compiled_sol['contracts'][contract_file][contract_name]['abi']
        bytecode = compiled_sol['contracts'][contract_file][contract_name]['evm']['bytecode']['object']
    except KeyError as e:
        print(f"Error extracting ABI and Bytecode for {contract_name}: {e}")
        return None, None

    return abi, bytecode

# Ensure the build directory exists
if not os.path.exists("build"):
    os.makedirs("build")

# List of contracts to compile
contracts_to_compile = [
    {"file": "../contracts/FLIncentiveMultiCat1.sol", "name": "FLIncentiveMultiCat1"},
    {"file": "../contracts/FLConsensusReputation.sol", "name": "FLConsensusReputation"}
]

# Compile each contract and save ABI and Bytecode
for contract in contracts_to_compile:
    abi, bytecode = compile_contract(contract["file"], contract["name"])
    if abi and bytecode:
        contract_basename = contract["name"]
        # Save ABI
        abi_path = f"../build/{contract_basename}_abi.json"
        with open(abi_path, "w") as abi_file:
            json.dump(abi, abi_file, indent=4)
        # Save Bytecode
        bytecode_path = f"../build/{contract_basename}_bytecode.txt"
        with open(bytecode_path, "w") as bytecode_file:
            bytecode_file.write(bytecode)
        print(f"{contract_basename} compiled successfully. ABI and Bytecode saved.")
    else:
        print(f"Failed to compile {contract['name']}.")

print("All contracts compiled.")


FLIncentiveMultiCat1 compiled successfully. ABI and Bytecode saved.
FLConsensusReputation compiled successfully. ABI and Bytecode saved.
All contracts compiled.


In [10]:
from web3 import Web3
from eth_account import Account
import json
import time

# -----------------------------
# Configuration and Setup
# -----------------------------

# Connect to Ganache CLI
ganache_url = "http://127.0.0.1:8545"  # Ensure Ganache CLI is running on this URL
w3 = Web3(Web3.HTTPProvider(ganache_url))
if not w3.is_connected():
    raise Exception("Failed to connect to Ganache")
print("Connected to Ganache\n")

# Load Private Keys from JSON
with open("private_keys.json", "r") as key_file:
    keys = json.load(key_file)

faucet_private_key = keys.get("faucet")
if not faucet_private_key:
    raise Exception("Faucet private key not found in private_keys.json")

faucet_account = w3.eth.account.from_key(faucet_private_key)
faucet_address = faucet_account.address
print(f"Faucet Account Address: {faucet_address}\n")

# -----------------------------
# Loading Contract ABIs and Bytecodes
# -----------------------------

def load_contract(name):
    """
    Loads the ABI and Bytecode for a given contract.
    """
    try:
        with open(f"../build/{name}_abi.json", "r") as abi_file:
            abi = json.load(abi_file)
        with open(f"../build/{name}_bytecode.txt", "r") as bytecode_file:
            bytecode = bytecode_file.read().strip()
        return abi, bytecode
    except FileNotFoundError as e:
        raise Exception(f"Error loading contract {name}: {e}")

# Load FLIncentiveMultiCat1
incentive_abi, incentive_bytecode = load_contract("FLIncentiveMultiCat1")

# Load FLConsensusReputation
consensus_abi, consensus_bytecode = load_contract("FLConsensusReputation")

# -----------------------------
# Contract Deployment Functions
# -----------------------------

def deploy_contract(abi, bytecode, constructor_args=()):
    """
    Deploys a smart contract to the blockchain.
    """
    try:
        contract = w3.eth.contract(abi=abi, bytecode=bytecode)
        # Build transaction
        tx = contract.constructor(*constructor_args).build_transaction({
            'from': faucet_address,
            'nonce': w3.eth.get_transaction_count(faucet_address),
            # 'gas': 500000,
            'gasPrice': w3.to_wei('20', 'gwei')
        })
        # Sign transaction
        signed_tx = w3.eth.account.sign_transaction(tx, faucet_private_key)
        # Send transaction
        tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
        print(f"Deploying contract... TxHash: {tx_hash.hex()}")
        # Wait for receipt
        tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Contract deployed at: {tx_receipt.contractAddress}\n")
        return tx_receipt.contractAddress
    except Exception as e:
        raise Exception(f"Error deploying contract: {e}")

# -----------------------------
# Deploy FLIncentiveMultiCat1 Contract
# -----------------------------

print("Deploying FLIncentiveMultiCat1 Contract...")
try:
    deadline = int(time.time()) + 86400  # Set deadline to 24 hours from now
    incentive_contract_address = deploy_contract(incentive_abi, incentive_bytecode, constructor_args=(deadline,))
except Exception as e:
    print(f"Error during FLIncentiveMultiCat1 deployment: {e}")
    incentive_contract_address = None

# -----------------------------
# Deploy FLConsensusReputation Contract
# -----------------------------

if incentive_contract_address:
    print("Deploying FLConsensusReputation Contract...")
    try:
        consensus_contract_address = deploy_contract(consensus_abi, consensus_bytecode, constructor_args=(incentive_contract_address,))
    except Exception as e:
        print(f"Error during FLConsensusReputation deployment: {e}")
        consensus_contract_address = None
else:
    print("Skipping FLConsensusReputation deployment due to missing incentive contract address.")
    consensus_contract_address = None

# -----------------------------
# Saving Deployed Addresses
# -----------------------------

if incentive_contract_address and consensus_contract_address:
    try:
        contract_addresses = {
            "FLIncentiveMultiCat1": incentive_contract_address,
            "FLConsensusReputation": consensus_contract_address
        }

        with open("contract_addresses.json", "w") as addr_file:
            json.dump(contract_addresses, addr_file, indent=4)

        print("Contract addresses saved to contract_addresses.json\n")
    except Exception as e:
        print(f"Error saving contract addresses: {e}")
else:
    print("One or more contract addresses are missing. Skipping address save.")


Connected to Ganache

Faucet Account Address: 0xCDe43483C86f50A17747d228fa0220D5019564b2

Deploying FLIncentiveMultiCat1 Contract...
Deploying contract... TxHash: 8aab030347984bea4c733274f70d3e61f95ee6eeb44bdadef88bafba143c5d70
Contract deployed at: 0xF6aE22bb84a096ef80E38E545f53b355Cf54E0f9

Deploying FLConsensusReputation Contract...
Error during FLConsensusReputation deployment: Error deploying contract: Incorrect argument count. Expected '0', got '1'.
One or more contract addresses are missing. Skipping address save.


In [12]:
from web3 import Web3
from eth_account import Account
import json
import time

# -----------------------------
# Configuration and Setup
# -----------------------------

# Connect to Ganache CLI
ganache_url = "http://127.0.0.1:8545"  # Ensure Ganache CLI is running on this URL
w3 = Web3(Web3.HTTPProvider(ganache_url))
if not w3.is_connected():
    raise Exception("Failed to connect to Ganache")
print("Connected to Ganache\n")

# Load Private Keys from JSON
with open("private_keys.json", "r") as key_file:
    keys = json.load(key_file)

faucet_private_key = keys.get("faucet")
if not faucet_private_key:
    raise Exception("Faucet private key not found in private_keys.json")

faucet_account = w3.eth.account.from_key(faucet_private_key)
faucet_address = faucet_account.address
print(f"Faucet Account Address: {faucet_address}\n")

# Ensure the faucet account has enough balance
def check_balance(address):
    balance = w3.eth.get_balance(address)
    print(f"Account {address} Balance: {w3.from_wei(balance, 'ether')} ETH")
    if balance < w3.to_wei(1, 'ether'):
        raise Exception("Insufficient funds in the faucet account")

check_balance(faucet_address)

# -----------------------------
# Loading Contract ABIs and Bytecodes
# -----------------------------

def load_contract(name):
    """
    Loads the ABI and Bytecode for a given contract.
    """
    try:
        with open(f"../build/{name}_abi.json", "r") as abi_file:
            abi = json.load(abi_file)
        with open(f"../build/{name}_bytecode.txt", "r") as bytecode_file:
            bytecode = bytecode_file.read().strip()
        return abi, bytecode
    except FileNotFoundError as e:
        raise Exception(f"Error loading contract {name}: {e}")

# Load FLIncentiveMultiCat1
incentive_abi, incentive_bytecode = load_contract("FLIncentiveMultiCat1")

# Load FLConsensusReputation
consensus_abi, consensus_bytecode = load_contract("FLConsensusReputation")

# -----------------------------
# Contract Deployment Functions
# -----------------------------

def deploy_contract(abi, bytecode, constructor_args=()):
    """
    Deploys a smart contract to the blockchain.
    """
    try:
        contract = w3.eth.contract(abi=abi, bytecode=bytecode)
        # Build transaction
        tx = contract.constructor(*constructor_args).build_transaction({
            'from': faucet_address,
            'nonce': w3.eth.get_transaction_count(faucet_address),
            # 'gas': 500000,
            # 'gasPrice': w3.to_wei('20', 'gwei'),
            # 'value': 0
        })
        # Sign transaction
        signed_tx = w3.eth.account.sign_transaction(tx, faucet_private_key)
        # Send transaction
        tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
        print(f"Deploying contract... TxHash: {tx_hash.hex()}")
        # Wait for receipt
        tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Contract deployed at: {tx_receipt.contractAddress}\n")
        return tx_receipt.contractAddress
    except Exception as e:
        raise Exception(f"Error deploying contract: {e}")

# -----------------------------
# Deploy FLIncentiveMultiCat1 Contract
# -----------------------------

print("Deploying FLIncentiveMultiCat1 Contract...")
try:
    deadline = int(time.time()) + 86400  # Set deadline to 24 hours from now
    incentive_contract_address = deploy_contract(incentive_abi, incentive_bytecode, constructor_args=(deadline,))
except Exception as e:
    print(f"Error during FLIncentiveMultiCat1 deployment: {e}")
    incentive_contract_address = None

# -----------------------------
# Deploy FLConsensusReputation Contract
# -----------------------------

if incentive_contract_address:
    print("Deploying FLConsensusReputation Contract...")
    try:
        consensus_contract_address = deploy_contract(consensus_abi, consensus_bytecode, constructor_args=(incentive_contract_address,))
    except Exception as e:
        print(f"Error during FLConsensusReputation deployment: {e}")
        consensus_contract_address = None
else:
    print("Skipping FLConsensusReputation deployment due to missing incentive contract address.")
    consensus_contract_address = None

# -----------------------------
# Saving Deployed Addresses
# -----------------------------

if incentive_contract_address and consensus_contract_address:
    try:
        contract_addresses = {
            "FLIncentiveMultiCat1": incentive_contract_address,
            "FLConsensusReputation": consensus_contract_address
        }

        with open("contract_addresses.json", "w") as addr_file:
            json.dump(contract_addresses, addr_file, indent=4)

        print("Contract addresses saved to contract_addresses.json\n")
    except Exception as e:
        print(f"Error saving contract addresses: {e}")
else:
    print("One or more contract addresses are missing. Skipping address save.")


Connected to Ganache

Faucet Account Address: 0xCDe43483C86f50A17747d228fa0220D5019564b2

Account 0xCDe43483C86f50A17747d228fa0220D5019564b2 Balance: 999.9709581 ETH
Deploying FLIncentiveMultiCat1 Contract...
Deploying contract... TxHash: 8322b9a2fcb0199498d72c0f75f610d0c70538e99eadbc10b33a5ed2b10a7244
Contract deployed at: 0x85A73a63f68a57583671f5800Fff98A57c8cDA0B

Deploying FLConsensusReputation Contract...
Error during FLConsensusReputation deployment: Error deploying contract: Incorrect argument count. Expected '0', got '1'.
One or more contract addresses are missing. Skipping address save.
